# Introduction

Before diving into the details of the problem, let's understand what is the goal?

```
1. We need to predict the short-term volatility (for 10 minutes range) for hundreds of stocks across different sectors. 
2. We have hundreds of millions of highly granular(i.e. detailed) data. 
3. For the first three months, the forecasting model will be over given dataset for 10 minute periods. For next three months, it will predicted on real time data. (Just like Jane street Market Prediction competition).
4. Evaluation Metrics: RMSPE (Root Mean Square Percentage Error).
5. This is a code competition and expect to predict 150K target values in RUN TIME.
6. Public and Private test data will have zero over lap. Hence, just build a generalized model/models. Forget about the leaderboard during training phase :)
```

# Dataset Overview

- Given `Order Book` snapshots and `Executed Trades` in parquet format.
- What are Order books and Executed Trades, refer to the notebook by organizers to get domain expertise.

# Domain Introduction and Resources

- Volatility: https://www.optiver.com/insights/guides/options-volatility/
- How Price is related to Volatility?: https://www.optiver.com/insights/guides/options-pricing/
- Organizer's Notebook on Introduction and Domain Knowledge: https://www.kaggle.com/jiashenliu/introduction-to-financial-concepts-and-data

In [ ]:
import os
import gc
import pickle
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from sklearn.model_selection import KFold

import xgboost as xgb

tqdm.pandas()
%matplotlib inline


import warnings
warnings.filterwarnings("ignore")

In [ ]:
# in order to preprocess parquet files, use the standard pyarrow lib
!pip install pyarrow

import pyarrow.parquet as pq

In [ ]:
# files
train = pd.read_csv("../input/optiver-realized-volatility-prediction/train.csv")
test = pd.read_csv("../input/optiver-realized-volatility-prediction/test.csv")
ss = pd.read_csv("../input/optiver-realized-volatility-prediction/sample_submission.csv")

print("Train Info: \n")
train.info()

print("\nTest Info: \n")
test.info()

print("\nSample Submission Format: \n")
ss.info()

# Data Attributes

- Train
    - stock_id: ID code for the stock. (as categorical in parquet files, can be converted into int8 dtype)
    - time_id : ID code for time bucket. Not necessarily sequential. Consistent across all stocks.
    - target  : The realized volatility computed over a `10 minute` window. For each stock-id-time_id pair, we have a target.
    

- Test
    - stock_id: same as above
    - time_id : same as above
    - row_id  : Unique identifier for the submission row. One row for each stock-time pair.


- Sample Submission
    - row_id  : same as above
    - target  : same as above

In [ ]:
train.head()

In [ ]:
train["stock_id"].value_counts()

So, for each stock ID, we have 3830/3829 time IDs, except stock 80, and stock 38, having 3820 and 3815 time IDs respectively. And we have total `112` stocks. Meta data for each stock are available in corresponding `book` and `trade` folders. We will look into each stock later, let's have a look on the time id, target distributions w.r.t. stock id.

In [ ]:
train["time_id"].value_counts()

As mentioned, yeah the time IDs are consistent throughout the stocks. P.S: There are 3830 time IDs. Let's pick a particular stock and have a brief overview of what we have.

In [ ]:
stock_example = train[train["stock_id"]==0]

plt.figure(figsize=(20, 8))
plt.plot(stock_example["time_id"].values, stock_example["target"].values, label="Target")
plt.plot(stock_example["time_id"].values, stock_example["target"].rolling(window=10).mean().values, label="Moving Avg WS10") #WS=Window Size
plt.title("Target values over time IDs for stock 0")
plt.legend()
plt.show()

In [ ]:
display(train["target"].describe())

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(20, 6))
train["target"].plot(kind="hist", bins=100, ax=axs[0], title="Target Distribution")
train["target"].progress_apply(np.log).plot(kind="hist", bins=100, ax=axs[1], title="log-transformed(for skewness) Target Distribution")

plt.figure(figsize=(20, 4))
plt.boxplot(train["target"], vert=False)
plt.title("Boxplot for Target")
plt.show()

In [ ]:
def book_length(stock_id):
    return len(pq.read_table(f"../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id={stock_id}"))
    pass

def trade_length(stock_id):
    return len(pq.read_table(f"../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id={stock_id}"))
    pass

train_meta = pd.DataFrame(columns=["stock_id", "len_order_book", "len_trade"])
train_meta["stock_id"] = train["stock_id"].unique()
train_meta["len_order_book"] = train_meta["stock_id"].progress_apply(book_length)
train_meta["len_trade"] = train_meta["stock_id"].progress_apply(trade_length)

train_meta

# Meta Data Overview

Let's investigate parquet files for a single stock for Order book and trades.

In [ ]:
# reading book sample
print("A book sample: \n")
book_sample = pq.read_table("../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0")
book_sample = book_sample.to_pandas()
display(book_sample)

# reading trade sample
print("\nA trade sample: \n")
trade_sample = pq.read_table("../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=0")
trade_sample = trade_sample.to_pandas()
display(trade_sample)

## Notes:

- Our data points are (X_i, Y_i), where X_i is the feature vector for every (stock_id, time_id) pair, and Y_i is the corresponding target.
- Y_i can be calculated from squared aggregation of log of WAP(weighted avg price), (Ref: Organizer's notebook.)
- If we create a cleaned, and one merged dataset, we can have below features (as of now, pure naive features) for every (X, Y):
    - Statistical data about `seconds_in_bucket`.
    - `wap` calculated from most and second most competitive buy/sell.
    - `log(wap)`, and probably the calculated squared aggregrate of log(wap) from the log return. Don't know whether we should use this as a feature in our model training, as it is most likely to have a high, very high correlation with the `target` value. Let's see, no sepculations now.
    - Aggregrate statistical values, and some feature engineering between pirces and sizes.

Speakng of `time_id`, let's see the occurances we have for a given time id per stock.

In [ ]:
# for book sample
print("For order book: \n", book_sample["time_id"].value_counts())
# for trade sample
print("For trade sample: \n", trade_sample["time_id"].value_counts())

We observed that, we have around ~3830 instances for a given `(stock_id, time_id)` pair, and for each pair we can have a seuqential data/array of WAP. The length may vary from 70-550 (it's for stock 0, results can be different for different stock ids). If we are to treat these as a sequential data, the length will be a matter of concern. Well, will dive into that later. 

Before diving into details, let's look into a single data point, i.e. one (stock_id, time_id) pair. Let's take `time_id=5` for `stock_id=0`.

In [ ]:
chunk_ob = book_sample[book_sample["time_id"]==5]                  # chun_ob means a chunk from order book


def plot_series(df, kind='price', plot_all=True, plot_wap=False):
    plt.figure(figsize=(20, 10))
    
    if plot_all:
        plt.plot(chunk_ob["seconds_in_bucket"], chunk_ob[f"bid_{kind}1"], label=f"bid_{kind}1")
        plt.plot(chunk_ob["seconds_in_bucket"], chunk_ob[f"ask_{kind}1"], label=f"ask_{kind}1")
        plt.plot(chunk_ob["seconds_in_bucket"], chunk_ob[f"bid_{kind}2"], label=f"bid_{kind}2")
        plt.plot(chunk_ob["seconds_in_bucket"], chunk_ob[f"ask_{kind}2"], label=f"ask_{kind}2")
    
    if plot_wap:
        plt.plot(chunk_ob["seconds_in_bucket"], chunk_ob[f"wap1"], label=f"wap1")
        plt.plot(chunk_ob["seconds_in_bucket"], chunk_ob[f"wap2"], label=f"wap2")
        
    plt.xlabel("seconds in bucket")
    plt.title(f"Buying/selling {kind} plot over seconds_in_bucket")
    plt.legend()
    pass

plot_series(chunk_ob)
plot_series(chunk_ob, kind="size")

Well, `ask_price2` and `bid_price1` seem to leading thier counterparts. Read that again.

For order size, nothing is interpretable from this graph for me. We will look into that later. As of now, the takeaway can be: there is no point in feeding all the buying/selling prices data for a (stock_id, time_id) pair. Cause it's redudant. What could be the relation with weighted average pricing? Let's see.

In [ ]:
# utilities for realized volatility calculation
# ref: https://www.kaggle.com/jiashenliu/introduction-to-financial-concepts-and-data/
def calculate_wap(df, rank="1"):
    df[f"wap{rank}"] = (df[f"bid_price{rank}"] * df[f"ask_size{rank}"] + df[f"ask_price{rank}"] * df[f"bid_size{rank}"])/(df[f"bid_size{rank}"] + df[f"ask_size{rank}"])
    return df
    pass

def calculate_logReturn(df, rank="1"):
    df.loc[:, f"log_return{rank}"] = np.log(df[f"wap{rank}"]).diff()
    return df[~df[f"log_return{rank}"].isnull()]
    pass

def calculate_volatility(logReturn):
    return np.sqrt(np.sum(logReturn**2))
    pass

# calculate wap1
calculate_wap(chunk_ob)
calculate_logReturn(chunk_ob)

# calculate wap2
calculate_wap(chunk_ob, rank="2")
calculate_logReturn(chunk_ob, rank="2")

plot_series(chunk_ob, plot_all=False, plot_wap=True)

In [ ]:
print("True target label for stock_id==0 and time_id==5: ", train[(train["stock_id"]==0) & (train["time_id"]==5)]["target"].values[0])
print("Rank-1 target label for stock_id==0 and time_id==5: ", calculate_volatility(chunk_ob["log_return1"]))
print("Rank-2 target label for stock_id==0 and time_id==5: ", calculate_volatility(chunk_ob["log_return2"]))
print("Avg of target label for stock_id==0 and time_id==5: ", (calculate_volatility(chunk_ob["log_return1"]) + calculate_volatility(chunk_ob["log_return2"]))/2)

Okay, now let's inspect trade data for the same `time_id`.

In [ ]:
chunk_trade = trade_sample[trade_sample["time_id"]==5]
chunk_trade.head()

So, what we need to find here is, how price is influenced by order_count and size. And the dependency of seconds_in_bucket. Keeping this simple, and considering the statistical features only, we can proceed for modeling.

# Feature Engineering and Dataset

Alright, we got a basic idea of what and how of the problem statement. Let's get on to it. There are actually pretty good baselines, so instead of describing more feature-centric approaches, I will build a XGB pipeline here. Let's go.

In [ ]:
# feature utils
def calculate_wap(df, rank="1"):
    """
    Weighted Average Pricing for a stock at a given time ID is given by:
    (bid_price1 * ask_size1 + bid_size1 * ask_price1)/(bid_size1 + ask_size1)

    It can further be extended to:

        sum(bid_price_i * ask_size_i + bid_size_i * ask_price_i)/sum(bid_size_i + ask_size_i)

    :param rank: which wap to calculate
    :param df: parquet table containing order book
    :return:
    """
    return (df[f"bid_price{rank}"] * df[f"ask_size{rank}"] + df[f"bid_size{rank}"] * df[f"ask_price{rank}"]) / (df[f"bid_size{rank}"] + df[f"ask_size{rank}"])


def calculate_logreturn(series):
    return np.log(series).diff()


def calculate_rv(series):
    return np.sqrt(np.sum(np.square(series)))


def count_unique(series):
    return len(np.unique(series))


def get_stats_window(df, seconds_in_bucket, features_dict, add_suffix=False):
    df_feature = df[df["seconds_in_bucket"] >= seconds_in_bucket].groupby(["time_id"]).agg(features_dict).reset_index()
    df_feature.columns = ["_".join(col) for col in df_feature.columns]
    
    if add_suffix:
        df_feature = df_feature.add_suffix("_" + str(seconds_in_bucket))
        
    return df_feature
    pass

In [ ]:
# configs
class cfg:
    
    paths = {
        # train path
        "train_csv": "../input/optiver-realized-volatility-prediction/train.csv",
        "train_book": "../input/optiver-realized-volatility-prediction/book_train.parquet",
        "train_trade": "../input/optiver-realized-volatility-prediction/trade_train.parquet",

        # test path
        "test_csv": "../input/optiver-realized-volatility-prediction/test.csv",
        "test_book": "../input/optiver-realized-volatility-prediction/book_test.parquet",
        "test_trade": "../input/optiver-realized-volatility-prediction/trade_test.parquet",
    }

    feature_dict_book = {
        "wap1": [np.sum, np.mean, np.std],
        "wap2": [np.sum, np.mean, np.std],
        "log_return1": [np.sum, calculate_rv, np.mean, np.std],
        "log_return2": [np.sum, calculate_rv, np.mean, np.std],
        "wap_balance": [np.sum, np.mean, np.std],
        "volume_imbalance": [np.sum, np.mean, np.std],
        "total_volume": [np.sum, np.mean, np.std],
        "price_spread1": [np.sum, np.mean, np.std],
        "price_spread2": [np.sum, np.mean, np.std],
        "bid_spread": [np.sum, np.mean, np.std],
        "ask_spread": [np.sum, np.mean, np.std],
    }

    feature_dict_trade = {
        "log_return": [calculate_rv],
        "seconds_in_bucket": [np.mean, count_unique],
        "size": [np.sum, np.mean],
        "order_count": [np.mean, np.sum],
        "agg_amount": [np.mean, np.sum, np.std]
    }
    
    model_params = {
        "xgb_bl": {
            "objective": "reg:squarederror",
            "booster": "gbtree",
            "nthread": -1,
            "eta": 0.3,
            "subsample": 0.2,
            "colsample_bytree": 0.33,
            "sampling_method": "uniform",
#             "tree_method": "gpu_hist"  # turn it on for GPU
        }
    }

In [ ]:
# order book features
def get_book_features(file_path):
    book_df = pd.read_parquet(file_path)

    # calculate wap
    book_df['wap1'] = calculate_wap(book_df, rank="1")
    book_df['wap2'] = calculate_wap(book_df, rank="2")

    # calculate log return
    book_df["log_return1"] = book_df.groupby(["time_id"])["wap1"].apply(calculate_logreturn)
    book_df["log_return2"] = book_df.groupby(["time_id"])["wap2"].apply(calculate_logreturn)

    # calculate balance
    book_df["wap_balance"] = abs(book_df["wap1"] - book_df["wap2"])
    book_df["volume_imbalance"] = abs(
        (book_df["ask_size1"] + book_df["ask_size2"]) - (book_df["bid_size1"] + book_df["bid_size2"]))
    book_df["total_volume"] = book_df["ask_size1"] + book_df["ask_size2"] + book_df["bid_size1"] + book_df[
        "bid_size2"]

    # calculate spread
    book_df["price_spread1"] = (book_df["ask_price1"] - book_df["bid_price1"]) / (
            (book_df["ask_price1"] + book_df["bid_price1"]) / 2)
    book_df["price_spread2"] = (book_df["ask_price2"] - book_df["bid_price2"]) / (
            (book_df["ask_price2"] + book_df["bid_price2"]) / 2)

    book_df["bid_spread"] = book_df["bid_price1"] - book_df["bid_price2"]
    book_df["ask_spread"] = book_df["ask_price1"] - book_df["ask_price2"]

    book_df_merged = get_stats_window(book_df, seconds_in_bucket=0, features_dict=cfg.feature_dict_book)

    book_df_450 = get_stats_window(book_df, seconds_in_bucket=450, features_dict=cfg.feature_dict_book, add_suffix=True)
    book_df_300 = get_stats_window(book_df, seconds_in_bucket=300, features_dict=cfg.feature_dict_book, add_suffix=True)
    book_df_150 = get_stats_window(book_df, seconds_in_bucket=150, features_dict=cfg.feature_dict_book, add_suffix=True)

    # merge stats
    book_df_merged = book_df_merged.merge(book_df_450, how="left", left_on="time_id_", right_on="time_id__450")
    book_df_merged = book_df_merged.merge(book_df_300, how="left", left_on="time_id_", right_on="time_id__300")
    book_df_merged = book_df_merged.merge(book_df_150, how="left", left_on="time_id_", right_on="time_id__150")


    book_df_merged.drop(columns=["time_id__450", "time_id__300", "time_id__150"], inplace=True)

    book_df_merged["row_id"] = book_df_merged["time_id_"].apply(lambda x: f"{file_path.split('=')[1]}-{x}")
    book_df_merged.drop(["time_id_"], axis=1, inplace=True)

    return book_df_merged

# trade features
def get_trade_features(file_path):
    trade_df = pd.read_parquet(file_path)
    
    trade_df["log_return"] = trade_df.groupby(["time_id"])["price"].apply(calculate_logreturn)
    trade_df["agg_amount"] = trade_df["price"] * trade_df["size"]/trade_df["order_count"]

    trade_df_merged = get_stats_window(trade_df, seconds_in_bucket=0, features_dict=cfg.feature_dict_trade)

    trade_df_450 = get_stats_window(trade_df, seconds_in_bucket=450, features_dict=cfg.feature_dict_trade, add_suffix=True)
    trade_df_300 = get_stats_window(trade_df, seconds_in_bucket=300, features_dict=cfg.feature_dict_trade, add_suffix=True)
    trade_df_150 = get_stats_window(trade_df, seconds_in_bucket=150, features_dict=cfg.feature_dict_trade, add_suffix=True)

    # merge stats
    trade_df_merged = trade_df_merged.merge(trade_df_450, how="left", left_on="time_id_", right_on="time_id__450")
    trade_df_merged = trade_df_merged.merge(trade_df_300, how="left", left_on="time_id_", right_on="time_id__300")
    trade_df_merged = trade_df_merged.merge(trade_df_150, how="left", left_on="time_id_", right_on="time_id__150")
    
    trade_df_merged.drop(columns=["time_id__450", "time_id__300", "time_id__150"], inplace=True)
    
    trade_df_merged = trade_df_merged.add_prefix("trade_")

    trade_df_merged["row_id"] = trade_df_merged["trade_time_id_"].apply(lambda x: f"{file_path.split('=')[1]}-{x}")
    trade_df_merged.drop(["trade_time_id_"], axis=1, inplace=True)

    return trade_df_merged

In [ ]:
class GetData:
    def __init__(self, df, book_path, trade_path):
        self.df = df.copy(deep=True)
        self.order_book_path = book_path
        self.trade_path = trade_path

        self._get_rowid()

    def _get_rowid(self):
        self.df["row_id"] = self.df["stock_id"].astype(str) + "-" + self.df["time_id"].astype(str)

    def get_time_stock(self):
        vol_cols = ['log_return1_calculate_rv', 'log_return2_calculate_rv',
                    'log_return1_calculate_rv_450', 'log_return2_calculate_rv_450',
                    'log_return1_calculate_rv_300', 'log_return2_calculate_rv_300',
                    'log_return1_calculate_rv_150', 'log_return2_calculate_rv_150',
                    'trade_log_return_calculate_rv', 'trade_log_return_calculate_rv_450',
                    'trade_log_return_calculate_rv_300', 'trade_log_return_calculate_rv_150']

        df_stock_id = self.df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min']).reset_index()
        df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
        df_stock_id = df_stock_id.add_suffix('_' + 'stock')

        df_time_id = self.df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min']).reset_index()
        df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
        df_time_id = df_time_id.add_suffix('_' + 'time')

        # Merge with original dataframe
        self.df = self.df.merge(df_stock_id, how='left', left_on=['stock_id'], right_on=['stock_id__stock'])
        self.df = self.df.merge(df_time_id, how='left', left_on=['time_id'], right_on=['time_id__time'])
        self.df.drop(['stock_id__stock', 'time_id__time'], axis=1, inplace=True)
        return self.df

    def process_features(self, list_stock_ids):
        def parallel_helper(stock_id):
            book_sample_path = os.path.join(self.order_book_path, f"stock_id={stock_id}")
            trade_sample_path = os.path.join(self.trade_path, f"stock_id={stock_id}")

            return pd.merge(get_book_features(book_sample_path), get_trade_features(trade_sample_path),
                            on="row_id",
                            how="left")

        df = Parallel(n_jobs=-1, verbose=1)(delayed(parallel_helper)(stock_id) for stock_id in list_stock_ids)
        df = pd.concat(df, ignore_index=True)

        return df

    def get_features(self):
        features_df = self.process_features(self.df["stock_id"].unique())
        self.df = self.df.merge(features_df, on=["row_id"], how="left")

        return self.get_time_stock()
        pass

In [ ]:
train_data = GetData(train, cfg.paths["train_book"], cfg.paths["train_trade"])
train_df = train_data.get_features()
train_df.head()

# Modeling

In [ ]:
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

def feval_rmspe(y_pred, xgb_dtrain):
    y_true = xgb_dtrain.get_label()
    return "RMSPE", rmspe(y_true, y_pred)

class TrainFer:
    def __init__(self, params_dict, n_splits, model_path, random_state=2021):
        self.params = params_dict
        self.n_splits = n_splits
        self.random_state = random_state
        self.model_path = model_path
        
        if not os.path.isdir(model_path):
            os.makedirs(model_path)
    
    def train(self, X, y):
        oof_predictions = np.zeros(X.shape[0])
        kfold = KFold(n_splits=self.n_splits, random_state=self.random_state, shuffle=True)
        
        for fold, (train_idx, val_idx) in enumerate(kfold.split(X)):
            print(f"\nFold - {fold}\n")

            x_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
            x_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
            
            x_train["stock_id"] = x_train["stock_id"].astype(int)
            x_val["stock_id"] = x_val["stock_id"].astype(int)

            dtrain = xgb.DMatrix(x_train, label=y_train, weight=1/np.square(y_train), enable_categorical=True)
            dval = xgb.DMatrix(x_val, label=y_val, weight=1/np.square(y_val), enable_categorical=True)

            model = xgb.train(self.params,
                              dtrain=dtrain,
                              num_boost_round=10000,
                              evals=[(dtrain, "dtrain"), (dval, "dval")],
                              verbose_eval=50,
                              feval=feval_rmspe,
                              early_stopping_rounds=200)
            
            pickle.dump(model, open(os.path.join(self.model_path, f"xgb_bl_{fold}.pkl"), "wb"))
            oof_predictions[val_idx] = model.predict(dval)
            
        rmspe_score = rmspe(y, oof_predictions)
        print(f"OOF RMSPE: {rmspe_score}")

In [ ]:
trainer = TrainFer(cfg.model_params["xgb_bl"], n_splits=5, model_path="./xgbBaseline/")
trainer.train(train_df.drop(columns=["row_id", "target", "time_id"]), train_df["target"])

### Work In Progress